<a href="https://colab.research.google.com/github/natanaelwgm/2025w-PromedUI-NLPCC-Ganjil20242025/blob/main/nlpcc_2025_week6_livecoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# 1. Install the OpenAI library (if not already installed in your Colab environment)
!pip install openai -q

# 2. Import necessary libraries
from openai import OpenAI
from google.colab import userdata # To access Colab secrets


In [8]:

# 3. Get the OpenAI API key from Colab secrets
try:
    openai_api_key = userdata.get('openai_key')
    if not openai_api_key:
        raise ValueError("The 'openai_key' secret was found but it is empty. Please ensure it has a value.")
except userdata.SecretNotFoundError:
    print("OpenAI API key not found in Colab secrets.")
    print("Please add your OpenAI API key as a secret named 'openai_key' in your Colab notebook.")
    print("You can do this by clicking on the key icon (🔑) in the left sidebar, then 'Add a new secret'.")
    openai_api_key = None # Ensure client initialization fails gracefully or is skipped
except Exception as e:
    print(f"An error occurred while trying to access the secret 'openai_key': {e}")
    openai_api_key = None

if openai_api_key:
    # 4. Initialize the OpenAI client
    # As per the provided documentation, the client is initialized like this.
    # If your API key is set as an environment variable OPENAI_API_KEY,
    # you can initialize with client = OpenAI()
    # But since we are fetching from Colab secrets, we pass it explicitly.
    client = OpenAI(api_key=openai_api_key)

    # 5. Define your single message (the "1 time chat")
    # We use the message format suitable for chat-like interactions,
    # which is a list of message objects.
    user_message_content = "Hi which one of Silfa's skills is aligned with this new job? the job is software engineer. List the 2 top most related and 2 least related from Silva's listed skills in CV"
    chat_input_messages = [
        {
            "role": "user",
            "content": user_message_content
        }
    ]

    print(f"Your message to ChatGPT: {user_message_content}\n")

    # 6. Make the API call to get the "1 time answer"
    # We use client.responses.create as shown in the provided documentation.
    # The model "gpt-4.1" is also taken from the documentation examples.
    try:
        response = client.responses.create(
            model="gpt-4o",  # Using the model specified in the provided docs
            input=chat_input_messages
        )

        # 7. Print the model's response
        # The response object has an 'output_text' attribute for the generated text.
        print("ChatGPT's Answer:")
        if response.output_text:
            print(response.output_text)
        else:
            print("The model did not return any text.")

    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        print("This could be due to an invalid API key, insufficient credits, model availability, or network issues.")
else:
    print("Cannot proceed without a valid OpenAI API key.")

Your message to ChatGPT: Hi which one of Silfa's skills is aligned with this new job? the job is software engineer. List the 2 top most related and 2 least related from Silva's listed skills in CV

ChatGPT's Answer:
Of course! Please share Silfa's skills from the CV, and I'll help identify which ones are most and least aligned with a software engineer position.


In [11]:
# 1. Install necessary libraries
!pip install openai pandas openpyxl -q

# 2. Import necessary libraries
from openai import OpenAI
from google.colab import userdata # To access Colab secrets
from google.colab import files # To handle file uploads
import pandas as pd
import io # To handle byte stream for pandas

In [10]:


# 3. Get the OpenAI API key from Colab secrets
try:
    openai_api_key = userdata.get('openai_key')
    if not openai_api_key:
        raise ValueError("The 'openai_key' secret was found but it is empty. Please ensure it has a value.")
except userdata.SecretNotFoundError:
    print("OpenAI API key not found in Colab secrets.")
    print("Please add your OpenAI API key as a secret named 'openai_key' in your Colab notebook.")
    print("You can do this by clicking on the key icon (🔑) in the left sidebar, then 'Add a new secret'.")
    openai_api_key = None
except Exception as e:
    print(f"An error occurred while trying to access the secret 'openai_key': {e}")
    openai_api_key = None

if openai_api_key:
    # 4. Initialize the OpenAI client
    client = OpenAI(api_key=openai_api_key)

    # 5. Upload and process the XLSX file
    print("Please upload an XLSX file:")
    uploaded = files.upload()

    dataframe_as_string = ""
    df = None

    if uploaded:
        file_name = next(iter(uploaded)) # Get the name of the uploaded file
        print(f"\nUploaded file: {file_name}")
        try:
            # Read the xlsx file into a pandas DataFrame
            df = pd.read_excel(io.BytesIO(uploaded[file_name]))
            print("\nDataFrame from the uploaded XLSX file:")
            print(df)

            # Convert DataFrame to a string representation to include in the prompt
            dataframe_as_string = df.to_string()
        except Exception as e:
            print(f"Error processing the Excel file: {e}")
            dataframe_as_string = "Error: Could not process the Excel file."
    else:
        print("\nNo file uploaded. Proceeding without Excel data.")
        dataframe_as_string = "No Excel data was provided."

    # 6. Define your single message, now including the DataFrame context
    user_question = "Hi which one of Silfa's skills is aligned with this new job? the job is software engineer. List the 2 top most related and 2 least related from Silva's listed skills in CV"

    # Construct the content for the API call
    if df is not None and not df.empty:
        prompt_content = f"Here is some data from an uploaded Excel file:\n\n```\n{dataframe_as_string}\n```\n\nMy question is: {user_question}"
    elif dataframe_as_string == "Error: Could not process the Excel file.":
         prompt_content = f"There was an issue processing an Excel file. The error was: {dataframe_as_string}\n\nMy question is: {user_question}"
    else: # No file uploaded or empty dataframe
        prompt_content = f"No Excel data was provided.\n\nMy question is: {user_question}"

    chat_input_messages = [
        {
            "role": "user",
            "content": prompt_content
        }
    ]

    print(f"\nYour message to ChatGPT (including data context if applicable):\n{prompt_content}\n")

    # 7. Make the API call to get the "1 time answer"
    try:
        response = client.responses.create(
            model="gpt-4o",  # Using the model specified in the provided docs
            input=chat_input_messages
        )

        # 8. Print the model's response
        print("ChatGPT's Answer:")
        if response.output_text:
            print(response.output_text)
        else:
            print("The model did not return any text.")

    except Exception as e:
        print(f"An error occurred during the API call: {e}")
        print("This could be due to an invalid API key, insufficient credits, model availability, or network issues.")
else:
    print("Cannot proceed without a valid OpenAI API key.")

Please upload an XLSX file:


Saving week6-skills.xlsx to week6-skills (1).xlsx

Uploaded file: week6-skills (1).xlsx

DataFrame from the uploaded XLSX file:
         Name                                             Skills
0       Silfa  Python Programming,Effective Communication,Gra...
1       Aulia  Data Analysis,Team Leadership,Fluent Spanish,V...
2       Salma  Cloud Computing (AWS/Azure),Critical Thinking,...
3  Bernadethe  Adobe Photoshop,Adaptability,Public Speaking,M...
4       Waran  Machine Learning,UX/UI Design,Empathy,Financia...

Your message to ChatGPT (including data context if applicable):
Here is some data from an uploaded Excel file:

```
         Name                                                                                               Skills
0       Silfa            Python Programming,Effective Communication,Graphic Design,Problem Solving,Time Management
1       Aulia                        Data Analysis,Team Leadership,Fluent Spanish,Video Editing,Project Management
2       Salma  Cloud

In [13]:
# Ensure you have the OpenAI library installed:
# !pip install openai

import os
import re
from openai import OpenAI

# For Google Colab secrets
# Make sure you have a secret named 'openai_key' in your Colab environment
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('openai_key')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found in Colab secrets. Please add it as 'openai_key'.")
    print("Successfully loaded OpenAI API key from Colab secrets.")
except ImportError:
    # Fallback for local execution (set an environment variable OPENAI_API_KEY)
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable or run in Colab with the secret.")
    print("Successfully loaded OpenAI API key from environment variable.")
except ValueError as e:
    print(f"Error: {e}")
    # You might want to exit or raise the error further depending on desired behavior
    # For this example, we'll let it proceed, but API calls will fail.
    OPENAI_API_KEY = "DUMMY_KEY_IF_NOT_FOUND" # Prevents crash if key is missing, API calls will fail

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)
MODEL_NAME = "gpt-4o" # Using the specified gpt-4o model

# --- Helper Function for OpenAI API Calls ---
def call_openai_api(user_prompt, system_prompt=None, model=MODEL_NAME):
    print(f"    📞 Calling OpenAI API (Model: {model})...")

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    print(f"    ✉️  Messages sent to API: {messages}")

    try:
        # Using client.responses.create as per the new OpenAI documentation
        response = client.responses.create(
            model=model,
            input=messages  # 'input' can take a list of messages
        )
        generated_text = response.output_text
        print(f"    ✅ Raw API Output (first 300 chars): '{generated_text[:300]}...'")
        return generated_text.strip()
    except Exception as e:
        print(f"    ❌ ERROR during OpenAI API call: {e}")
        import traceback
        traceback.print_exc()
        return None

# --- Helper Function for Parsing Numbered Lists ---
def parse_numbered_list_from_llm(llm_output_str, expected_items=None):
    print(f"    📄 Attempting to parse LLM output: '{llm_output_str[:100]}...'")
    items = []
    if not llm_output_str:
        print("    ⚠️ LLM output is empty. Cannot parse.")
        return []

    lines = llm_output_str.strip().split('\n')
    for line in lines:
        line = line.strip()
        # Remove leading numbers/bullets (e.g., "1.", "  2)", "- ", "* ")
        cleaned_line = re.sub(r"^\s*[\d\.\-\*\)]+\s*", "", line).strip()
        if cleaned_line: # Add only if there's content after stripping
            items.append(cleaned_line)

    if expected_items is not None and len(items) != expected_items:
        print(f"    ⚠️ Warning: Expected {expected_items} items, but parsed {len(items)}. Items: {items}")
        # If parsing fails to get the exact number, but we got something, return it.
        # If it's critical, the calling agent should handle it.

    if not items and llm_output_str: # Fallback if parsing yields nothing but there was input
        print("    ⚠️ Standard parsing failed to extract items, but LLM output was not empty. Returning raw output as a single item list.")
        return [llm_output_str.strip()]

    print(f"    📊 Parsed items: {items}")
    return items

# --- Agent Definitions ---

def agent_planner(topic):
    print("\n🚀 --- Agent: Planner [STARTING] --- 🚀")
    print(f"🎯 Input to Planner: Essay Topic = '{topic}'")

    system_prompt = "You are an expert essay planner. Your task is to identify key arguments."
    user_prompt = (
        f"For an essay on the topic: '{topic}', please generate exactly 3 distinct main arguments. "
        "Each argument should be a single, concise sentence. "
        "Present these arguments as a numbered list, with each argument on a new line. For example:\n"
        "1. First argument.\n"
        "2. Second argument.\n"
        "3. Third argument."
    )

    raw_output = call_openai_api(user_prompt, system_prompt)

    if raw_output:
        arguments = parse_numbered_list_from_llm(raw_output, expected_items=3)
        if len(arguments) == 3:
            print(f"✅ Output from Planner (processed): {arguments}")
            print("🚀 --- Agent: Planner [FINISHED] --- 🚀")
            return arguments
        else:
            print(f"❌ Error: Planner did not return 3 arguments. Got {len(arguments)}: {arguments}")
            print("🚀 --- Agent: Planner [FINISHED WITH ERROR] --- 🚀")
            return None # Or handle error more gracefully
    else:
        print("❌ Error: Planner received no output from API.")
        print("🚀 --- Agent: Planner [FINISHED WITH ERROR] --- 🚀")
        return None

def agent_argument_paragraph_maker(topic, argument, arg_num):
    print(f"\n📝 --- Agent: Argument Paragraph Maker (Argument {arg_num}) [STARTING] --- 📝")
    print(f"🎯 Input: Topic='{topic}', Argument='{argument}'")

    system_prompt = "You are an expert essay writer. Your task is to write a well-developed paragraph for a given argument."
    user_prompt = (
        f"I am writing an essay on the topic: '{topic}'. "
        f"Please write a detailed paragraph for argument number {arg_num}. The argument is: '{argument}'. "
        "The paragraph should elaborate on this argument, provide supporting details or examples, and be well-structured. "
        "Aim for approximately 4-6 sentences."
    )

    paragraph = call_openai_api(user_prompt, system_prompt)

    if paragraph:
        print(f"✅ Output from Argument Paragraph Maker (Argument {arg_num}):\n{paragraph}")
    else:
        print(f"❌ Error: Argument Paragraph Maker (Argument {arg_num}) received no output.")
    print(f"📝 --- Agent: Argument Paragraph Maker (Argument {arg_num}) [FINISHED] --- 📝")
    return paragraph

def agent_opening_paragraph_maker(topic, arguments_list):
    print("\n🌅 --- Agent: Opening Paragraph Maker [STARTING] --- 🌅")
    print(f"🎯 Input: Topic='{topic}', Main Arguments='{arguments_list}'")

    if not arguments_list or len(arguments_list) != 3:
        print("❌ Error: Opening Paragraph Maker requires 3 main arguments.")
        print("🌅 --- Agent: Opening Paragraph Maker [FINISHED WITH ERROR] --- 🌅")
        return None

    system_prompt = "You are an expert essay writer. Your task is to craft compelling introduction paragraphs."
    user_prompt = (
        f"Write an engaging opening paragraph for an essay on the topic: '{topic}'. "
        "This essay will explore the following three main arguments:\n"
        f"1. {arguments_list[0]}\n"
        f"2. {arguments_list[1]}\n"
        f"3. {arguments_list[2]}\n"
        "The opening paragraph should introduce the topic, provide brief context, and clearly state the essay's thesis "
        "or outline the main arguments that will be discussed. Do not write the full essay, only the opening paragraph."
    )

    opening_paragraph = call_openai_api(user_prompt, system_prompt)

    if opening_paragraph:
        print(f"✅ Output from Opening Paragraph Maker:\n{opening_paragraph}")
    else:
        print("❌ Error: Opening Paragraph Maker received no output.")
    print("🌅 --- Agent: Opening Paragraph Maker [FINISHED] --- 🌅")
    return opening_paragraph

def agent_closing_paragraph_maker(topic, arguments_list):
    print("\n🌇 --- Agent: Closing Paragraph Maker [STARTING] --- 🌇")
    print(f"🎯 Input: Topic='{topic}', Main Arguments='{arguments_list}'")

    if not arguments_list or len(arguments_list) != 3:
        print("❌ Error: Closing Paragraph Maker requires 3 main arguments.")
        print("🌇 --- Agent: Closing Paragraph Maker [FINISHED WITH ERROR] --- 🌇")
        return None

    system_prompt = "You are an expert essay writer. Your task is to craft impactful concluding paragraphs."
    user_prompt = (
        f"Write a strong concluding paragraph for an essay on the topic: '{topic}'. "
        "The essay has already discussed the following three main arguments:\n"
        f"1. {arguments_list[0]}\n"
        f"2. {arguments_list[1]}\n"
        f"3. {arguments_list[2]}\n"
        "The concluding paragraph should summarize the key points made in the essay, restate the thesis in a new way, "
        "and offer a final thought, implication, or call to action. Do not introduce new arguments. "
        "Do not write the full essay, only the closing paragraph."
    )

    closing_paragraph = call_openai_api(user_prompt, system_prompt)

    if closing_paragraph:
        print(f"✅ Output from Closing Paragraph Maker:\n{closing_paragraph}")
    else:
        print("❌ Error: Closing Paragraph Maker received no output.")
    print("🌇 --- Agent: Closing Paragraph Maker [FINISHED] --- 🌇")
    return closing_paragraph

def agent_flow_improver(topic):
    print("\n🔗 --- Agent: Flow Improver (Transitions) [STARTING] --- 🔗")
    print(f"🎯 Input: Topic='{topic}'")

    system_prompt = "You are an expert in academic writing, specializing in creating smooth transitions between essay paragraphs."
    user_prompt = (
        f"For an essay on the topic: '{topic}', which will consist of an introduction, three main body paragraphs "
        "(each discussing a distinct argument), and a conclusion, please generate four specific transition sentences:\n"
        "1. A sentence to smoothly connect the end of the introduction to the beginning of the first argument paragraph.\n"
        "2. A sentence to transition from the end of the first argument paragraph to the beginning of the second argument paragraph.\n"
        "3. A sentence to transition from the end of the second argument paragraph to the beginning of the third argument paragraph.\n"
        "4. A sentence to smoothly connect the end of the third argument paragraph to the beginning of the conclusion.\n"
        "Present these transition sentences as a numbered list, with each transition on a new line. For example:\n"
        "1. Transition sentence one.\n"
        "2. Transition sentence two.\n"
        "..."
    )

    raw_output = call_openai_api(user_prompt, system_prompt)

    if raw_output:
        transitions = parse_numbered_list_from_llm(raw_output, expected_items=4)
        if len(transitions) == 4:
            print(f"✅ Output from Flow Improver (processed): {transitions}")
            print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED] --- 🔗")
            return transitions
        else:
            print(f"❌ Error: Flow Improver did not return 4 transitions. Got {len(transitions)}: {transitions}")
            print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED WITH ERROR] --- 🔗")
            return None
    else:
        print("❌ Error: Flow Improver received no output from API.")
        print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED WITH ERROR] --- 🔗")
        return None

# --- Essay Combiner (Not an AI agent, just a Python function) ---
def agent_essay_combiner(opening_para, arg_paras_list, closing_para, transitions_list):
    print("\n🧩 --- Agent: Essay Combiner [STARTING] --- 🧩")
    if not all([opening_para, arg_paras_list, closing_para, transitions_list]):
        print("❌ Error: Missing one or more parts for essay combination.")
        print(f"Debug Info: Opening: {bool(opening_para)}, Args: {bool(arg_paras_list)}, Closing: {bool(closing_para)}, Transitions: {bool(transitions_list)}")
        if arg_paras_list: print(f"Arg Paras Length: {len(arg_paras_list)}")
        if transitions_list: print(f"Transitions Length: {len(transitions_list)}")
        print("🧩 --- Agent: Essay Combiner [FINISHED WITH ERROR] --- 🧩")
        return "Error: Could not combine essay due to missing parts."

    if len(arg_paras_list) != 3:
        print(f"❌ Error: Expected 3 argument paragraphs, got {len(arg_paras_list)}.")
        print("🧩 --- Agent: Essay Combiner [FINISHED WITH ERROR] --- 🧩")
        return "Error: Incorrect number of argument paragraphs."

    if len(transitions_list) != 4:
        print(f"❌ Error: Expected 4 transition sentences, got {len(transitions_list)}.")
        print("🧩 --- Agent: Essay Combiner [FINISHED WITH ERROR] --- 🧩")
        return "Error: Incorrect number of transition sentences."

    full_essay = f"{opening_para}\n\n"
    full_essay += f"{transitions_list[0]}\n\n"
    full_essay += f"{arg_paras_list[0]}\n\n"
    full_essay += f"{transitions_list[1]}\n\n"
    full_essay += f"{arg_paras_list[1]}\n\n"
    full_essay += f"{transitions_list[2]}\n\n"
    full_essay += f"{arg_paras_list[2]}\n\n"
    full_essay += f"{transitions_list[3]}\n\n"
    full_essay += f"{closing_para}"

    print("✅ Essay combined successfully.")
    print("🧩 --- Agent: Essay Combiner [FINISHED] --- 🧩")
    return full_essay

# --- Main Agentic Flow Execution ---
def run_essay_maker_flow(topic):
    print(f"\n🌟🌟🌟 STARTING ESSAY MAKER FLOW for topic: '{topic}' 🌟🌟🌟")

    # 1. Agent Planner
    main_arguments = agent_planner(topic)
    if not main_arguments or len(main_arguments) != 3:
        print("🛑 CRITICAL ERROR: Planner failed. Aborting essay creation.")
        return "Essay generation failed: Planner did not produce 3 arguments."

    # 2. Agent Opening Paragraph Maker
    opening_paragraph = agent_opening_paragraph_maker(topic, main_arguments)
    if not opening_paragraph:
        print("🛑 CRITICAL ERROR: Opening Paragraph Maker failed. Aborting essay creation.")
        return "Essay generation failed: Could not create opening paragraph."

    # 3. Agent Argument Paragraph Makers
    argument_paragraphs = []
    for i, arg_text in enumerate(main_arguments):
        p_num = i + 1
        paragraph = agent_argument_paragraph_maker(topic, arg_text, p_num)
        if not paragraph:
            print(f"🛑 CRITICAL ERROR: Argument Paragraph Maker for arg {p_num} failed. Aborting essay creation.")
            return f"Essay generation failed: Could not create paragraph for argument {p_num}."
        argument_paragraphs.append(paragraph)

    if len(argument_paragraphs) != 3:
        print("🛑 CRITICAL ERROR: Did not generate 3 argument paragraphs. Aborting essay creation.")
        return "Essay generation failed: Incorrect number of argument paragraphs generated."

    # 4. Agent Flow Improver (Transitions)
    transitions = agent_flow_improver(topic)
    if not transitions or len(transitions) != 4:
        print("🛑 CRITICAL ERROR: Flow Improver failed to generate 4 transitions. Aborting essay creation.")
        return "Essay generation failed: Could not create paragraph transitions."

    # 5. Agent Closing Paragraph Maker
    closing_paragraph = agent_closing_paragraph_maker(topic, main_arguments)
    if not closing_paragraph:
        print("🛑 CRITICAL ERROR: Closing Paragraph Maker failed. Aborting essay creation.")
        return "Essay generation failed: Could not create closing paragraph."

    # 6. Agent Essay Combiner
    final_essay = agent_essay_combiner(opening_paragraph, argument_paragraphs, closing_paragraph, transitions)

    print("\n🌟🌟🌟 ESSAY MAKER FLOW COMPLETED 🌟🌟🌟")
    return final_essay

# --- Example Usage ---
if __name__ == "__main__":
    # This is the "Input dari manusia: topik buat essay"
    # For Colab, you might want to use an input field:
    # user_topic = input("Please enter the topic for your essay: ")
    user_topic = "Hari Minggu ada CFD di Depok"
    # user_topic = "The impact of social media on modern society"
    # user_topic = "The future of artificial intelligence in healthcare"

    print(f"User input essay topic: '{user_topic}'")

    # Check if API key was loaded, otherwise skip API calls
    if OPENAI_API_KEY == "DUMMY_KEY_IF_NOT_FOUND" or not OPENAI_API_KEY:
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! OpenAI API Key is missing or invalid.                  !!!")
        print("!!! The script will not call the OpenAI API.               !!!")
        print("!!! Please ensure 'openai_key' is set in Colab secrets   !!!")
        print("!!! or OPENAI_API_KEY environment variable is set.         !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        final_essay_output = "Error: OpenAI API Key not configured. Essay generation skipped."
    else:
        # Run the agentic flow
        final_essay_output = run_essay_maker_flow(user_topic)

    # This is the "1 time answer to chatgpt" (i.e., the final output of the flow)
    print("\n\n📜📜📜 --- FINAL ESSAY --- 📜📜📜")
    print(final_essay_output)
    print("\n📜📜📜 --- END OF ESSAY --- 📜📜📜")

Successfully loaded OpenAI API key from Colab secrets.
User input essay topic: 'Hari Minggu ada CFD di Depok'

🌟🌟🌟 STARTING ESSAY MAKER FLOW for topic: 'Hari Minggu ada CFD di Depok' 🌟🌟🌟

🚀 --- Agent: Planner [STARTING] --- 🚀
🎯 Input to Planner: Essay Topic = 'Hari Minggu ada CFD di Depok'
    📞 Calling OpenAI API (Model: gpt-4o)...
    ✉️  Messages sent to API: [{'role': 'system', 'content': 'You are an expert essay planner. Your task is to identify key arguments.'}, {'role': 'user', 'content': "For an essay on the topic: 'Hari Minggu ada CFD di Depok', please generate exactly 3 distinct main arguments. Each argument should be a single, concise sentence. Present these arguments as a numbered list, with each argument on a new line. For example:\n1. First argument.\n2. Second argument.\n3. Third argument."}]
    ✅ Raw API Output (first 300 chars): '1. CFD in Depok on Sundays promotes a healthier lifestyle by encouraging physical activities like walking, jogging, and cycling.
2. The even

In [14]:
# Step 1: Install necessary packages (run this cell first in Colab)
!pip install openai pypandoc python-docx --quiet
!apt-get install pandoc -y --quiet
print("OpenAI, pypandoc, python-docx, and pandoc installed/updated.")

import os
import re
from openai import OpenAI
import pypandoc # For Markdown to DOCX conversion

# For Google Colab secrets
# Make sure you have a secret named 'openai_key' in your Colab environment
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('openai_key')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found in Colab secrets. Please add it as 'openai_key'.")
    print("Successfully loaded OpenAI API key from Colab secrets.")
except ImportError:
    # Fallback for local execution (set an environment variable OPENAI_API_KEY)
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable or run in Colab with the secret.")
    print("Successfully loaded OpenAI API key from environment variable.")
except ValueError as e:
    print(f"Error: {e}")
    OPENAI_API_KEY = "DUMMY_KEY_IF_NOT_FOUND"

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)
MODEL_NAME = "gpt-4o"

# --- Helper Function for OpenAI API Calls ---
def call_openai_api(user_prompt, system_prompt=None, model=MODEL_NAME):
    print(f"    📞 Calling OpenAI API (Model: {model})...")

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    print(f"    ✉️  Messages sent to API: {messages}")

    try:
        response = client.responses.create(
            model=model,
            input=messages
        )
        generated_text = response.output_text
        print(f"    ✅ Raw API Output (first 300 chars): '{generated_text[:300]}...'")
        return generated_text.strip()
    except Exception as e:
        print(f"    ❌ ERROR during OpenAI API call: {e}")
        import traceback
        traceback.print_exc()
        return None

# --- Helper Function for Parsing Numbered Lists ---
def parse_numbered_list_from_llm(llm_output_str, expected_items=None):
    print(f"    📄 Attempting to parse LLM output: '{llm_output_str[:100]}...'")
    items = []
    if not llm_output_str:
        print("    ⚠️ LLM output is empty. Cannot parse.")
        return []

    lines = llm_output_str.strip().split('\n')
    for line in lines:
        line = line.strip()
        cleaned_line = re.sub(r"^\s*[\d\.\-\*\)]+\s*", "", line).strip()
        if cleaned_line:
            items.append(cleaned_line)

    if expected_items is not None and len(items) != expected_items:
        print(f"    ⚠️ Warning: Expected {expected_items} items, but parsed {len(items)}. Items: {items}")

    if not items and llm_output_str:
        print("    ⚠️ Standard parsing failed to extract items, but LLM output was not empty. Returning raw output as a single item list.")
        return [llm_output_str.strip()]

    print(f"    📊 Parsed items: {items}")
    return items

# --- Agent Definitions ---

def agent_planner(topic):
    print("\n🚀 --- Agent: Planner [STARTING] --- 🚀")
    print(f"🎯 Input to Planner: Essay Topic = '{topic}'")

    system_prompt = "You are an expert essay planner. Your task is to identify key arguments."
    user_prompt = (
        f"For an essay on the topic: '{topic}', please generate exactly 3 distinct main arguments. "
        "Each argument should be a single, concise sentence. "
        "Present these arguments as a numbered list, with each argument on a new line. For example:\n"
        "1. First argument.\n"
        "2. Second argument.\n"
        "3. Third argument."
    )

    raw_output = call_openai_api(user_prompt, system_prompt)

    if raw_output:
        arguments = parse_numbered_list_from_llm(raw_output, expected_items=3)
        if len(arguments) == 3:
            print(f"✅ Output from Planner (processed): {arguments}")
            print("🚀 --- Agent: Planner [FINISHED] --- 🚀")
            return arguments
        else:
            print(f"❌ Error: Planner did not return 3 arguments. Got {len(arguments)}: {arguments}")
            print("🚀 --- Agent: Planner [FINISHED WITH ERROR] --- 🚀")
            # Fallback: if we got *some* arguments, try to use the first 3, or pad if fewer.
            # For simplicity here, we'll stick to requiring exactly 3.
            if len(arguments) > 0:
                 print(f"⚠️ Planner returned {len(arguments)} arguments, using the first 3 if available or padding.")
                 return (arguments + [f"Placeholder argument {i+1}" for i in range(len(arguments), 3)])[:3]
            return None
    else:
        print("❌ Error: Planner received no output from API.")
        print("🚀 --- Agent: Planner [FINISHED WITH ERROR] --- 🚀")
        return None

def agent_argument_paragraph_maker(topic, argument, arg_num):
    print(f"\n📝 --- Agent: Argument Paragraph Maker (Argument {arg_num}) [STARTING] --- 📝")
    print(f"🎯 Input: Topic='{topic}', Argument='{argument}'")

    system_prompt = "You are an expert essay writer. Your task is to write a well-developed paragraph for a given argument."
    user_prompt = (
        f"I am writing an essay on the topic: '{topic}'. "
        f"Please write a detailed paragraph for argument number {arg_num}. The argument is: '{argument}'. "
        "The paragraph should elaborate on this argument, provide supporting details or examples, and be well-structured. "
        "Aim for approximately 4-6 sentences. Ensure the paragraph is written in clear, formal essay style."
    )

    paragraph = call_openai_api(user_prompt, system_prompt)

    if paragraph:
        print(f"✅ Output from Argument Paragraph Maker (Argument {arg_num}):\n{paragraph}")
    else:
        print(f"❌ Error: Argument Paragraph Maker (Argument {arg_num}) received no output.")
    print(f"📝 --- Agent: Argument Paragraph Maker (Argument {arg_num}) [FINISHED] --- 📝")
    return paragraph

def agent_opening_paragraph_maker(topic, arguments_list):
    print("\n🌅 --- Agent: Opening Paragraph Maker [STARTING] --- 🌅")
    print(f"🎯 Input: Topic='{topic}', Main Arguments='{arguments_list}'")

    if not arguments_list or len(arguments_list) != 3:
        print("❌ Error: Opening Paragraph Maker requires 3 main arguments.")
        print("🌅 --- Agent: Opening Paragraph Maker [FINISHED WITH ERROR] --- 🌅")
        return None

    system_prompt = "You are an expert essay writer. Your task is to craft compelling introduction paragraphs."
    user_prompt = (
        f"Write an engaging opening paragraph for an essay on the topic: '{topic}'. "
        "This essay will explore the following three main arguments:\n"
        f"1. {arguments_list[0]}\n"
        f"2. {arguments_list[1]}\n"
        f"3. {arguments_list[2]}\n"
        "The opening paragraph should introduce the topic, provide brief context, and clearly state the essay's thesis "
        "or outline the main arguments that will be discussed. Do not write the full essay, only the opening paragraph. "
        "Ensure the paragraph is written in clear, formal essay style."
    )

    opening_paragraph = call_openai_api(user_prompt, system_prompt)

    if opening_paragraph:
        print(f"✅ Output from Opening Paragraph Maker:\n{opening_paragraph}")
    else:
        print("❌ Error: Opening Paragraph Maker received no output.")
    print("🌅 --- Agent: Opening Paragraph Maker [FINISHED] --- 🌅")
    return opening_paragraph

def agent_closing_paragraph_maker(topic, arguments_list):
    print("\n🌇 --- Agent: Closing Paragraph Maker [STARTING] --- 🌇")
    print(f"🎯 Input: Topic='{topic}', Main Arguments='{arguments_list}'")

    if not arguments_list or len(arguments_list) != 3:
        print("❌ Error: Closing Paragraph Maker requires 3 main arguments.")
        print("🌇 --- Agent: Closing Paragraph Maker [FINISHED WITH ERROR] --- 🌇")
        return None

    system_prompt = "You are an expert essay writer. Your task is to craft impactful concluding paragraphs."
    user_prompt = (
        f"Write a strong concluding paragraph for an essay on the topic: '{topic}'. "
        "The essay has already discussed the following three main arguments:\n"
        f"1. {arguments_list[0]}\n"
        f"2. {arguments_list[1]}\n"
        f"3. {arguments_list[2]}\n"
        "The concluding paragraph should summarize the key points made in the essay, restate the thesis in a new way, "
        "and offer a final thought, implication, or call to action. Do not introduce new arguments. "
        "Do not write the full essay, only the closing paragraph. Ensure the paragraph is written in clear, formal essay style."
    )

    closing_paragraph = call_openai_api(user_prompt, system_prompt)

    if closing_paragraph:
        print(f"✅ Output from Closing Paragraph Maker:\n{closing_paragraph}")
    else:
        print("❌ Error: Closing Paragraph Maker received no output.")
    print("🌇 --- Agent: Closing Paragraph Maker [FINISHED] --- 🌇")
    return closing_paragraph

def agent_flow_improver(topic):
    print("\n🔗 --- Agent: Flow Improver (Transitions) [STARTING] --- 🔗")
    print(f"🎯 Input: Topic='{topic}'")

    system_prompt = "You are an expert in academic writing, specializing in creating smooth and natural transition sentences between essay paragraphs."
    user_prompt = (
        f"For an essay on the topic: '{topic}', which will consist of an introduction, three main body paragraphs "
        "(each discussing a distinct argument), and a conclusion, please generate four specific, natural-sounding transition sentences:\n"
        "1. A sentence to smoothly connect the end of the introduction to the beginning of the first argument paragraph.\n"
        "2. A sentence to transition from the end of the first argument paragraph to the beginning of the second argument paragraph.\n"
        "3. A sentence to transition from the end of the second argument paragraph to the beginning of the third argument paragraph.\n"
        "4. A sentence to smoothly connect the end of the third argument paragraph to the beginning of the conclusion.\n"
        "Present these transition sentences as a numbered list, with each transition on a new line. Each should be a single, complete sentence. For example:\n"
        "1. Transition sentence one.\n"
        "2. Transition sentence two.\n"
        "..."
    )

    raw_output = call_openai_api(user_prompt, system_prompt)

    if raw_output:
        transitions = parse_numbered_list_from_llm(raw_output, expected_items=4)
        if len(transitions) == 4:
            print(f"✅ Output from Flow Improver (processed): {transitions}")
            print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED] --- 🔗")
            return transitions
        else:
            print(f"❌ Error: Flow Improver did not return 4 transitions. Got {len(transitions)}: {transitions}")
            print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED WITH ERROR] --- 🔗")
            if len(transitions) > 0:
                 print(f"⚠️ Flow Improver returned {len(transitions)} transitions, using what's available or padding.")
                 return (transitions + [f"Placeholder transition {i+1}" for i in range(len(transitions), 4)])[:4]
            return None
    else:
        print("❌ Error: Flow Improver received no output from API.")
        print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED WITH ERROR] --- 🔗")
        return None

# --- Essay Combiner & Markdown Formatter ---
def agent_essay_combiner_markdown(topic, opening_para, arg_paras_list, closing_para, transitions_list):
    print("\n🧩 --- Agent: Essay Combiner & Markdown Formatter [STARTING] --- 🧩")

    # Basic validation
    if not all([topic, opening_para, arg_paras_list, closing_para, transitions_list]):
        print("❌ Error: Missing one or more parts for essay combination.")
        print(f"Debug Info: Topic: {bool(topic)}, Opening: {bool(opening_para)}, Args: {bool(arg_paras_list)}, Closing: {bool(closing_para)}, Transitions: {bool(transitions_list)}")
        if arg_paras_list: print(f"Arg Paras Length: {len(arg_paras_list)}")
        if transitions_list: print(f"Transitions Length: {len(transitions_list)}")
        print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED WITH ERROR] --- 🧩")
        return "Error: Could not combine essay due to missing parts."

    if len(arg_paras_list) != 3:
        print(f"❌ Error: Expected 3 argument paragraphs, got {len(arg_paras_list)}.")
        print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED WITH ERROR] --- 🧩")
        return "Error: Incorrect number of argument paragraphs."

    if len(transitions_list) != 4:
        print(f"❌ Error: Expected 4 transition sentences, got {len(transitions_list)}.")
        print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED WITH ERROR] --- 🧩")
        return "Error: Incorrect number of transition sentences."

    # Create a title from the topic
    # Simple title case, or you can ask an LLM for a more creative title
    essay_title = topic.title()

    markdown_essay = f"# {essay_title}\n\n"
    markdown_essay += f"{opening_para}\n\n"

    # Interleave argument paragraphs and transitions
    markdown_essay += f"{transitions_list[0]}\n\n"
    markdown_essay += f"{arg_paras_list[0]}\n\n"

    markdown_essay += f"{transitions_list[1]}\n\n"
    markdown_essay += f"{arg_paras_list[1]}\n\n"

    markdown_essay += f"{transitions_list[2]}\n\n"
    markdown_essay += f"{arg_paras_list[2]}\n\n"

    markdown_essay += f"{transitions_list[3]}\n\n"
    markdown_essay += f"{closing_para}"

    print("✅ Essay combined and formatted in Markdown successfully.")
    print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED] --- 🧩")
    return markdown_essay

# --- DOCX Export Function ---
def export_to_docx(markdown_content, filename="essay.docx"):
    print(f"\n📄 --- Exporting to DOCX: {filename} [STARTING] --- 📄")
    if not markdown_content or markdown_content.startswith("Error:"):
        print("❌ Error: Markdown content is invalid or indicates a prior error. Skipping DOCX export.")
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH ERROR] --- 📄")
        return False

    try:
        # Ensure Pandoc is available (pypandoc raises an error if not)
        pypandoc.ensure_pandoc_installed(quiet=True)

        output = pypandoc.convert_text(
            markdown_content,
            'docx',
            format='md',
            outputfile=filename
        )
        assert output == "" # convert_text returns an empty string on success when outputfile is specified
        print(f"✅ Successfully exported essay to {filename}")
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED] --- 📄")
        print(f"ℹ️  In Google Colab, you can find '{filename}' in the file browser on the left panel and download it.")
        return True
    except OSError as e: # OSError is raised by pypandoc if pandoc is not found
        print(f"❌ ERROR: Pandoc not found or not configured correctly. Cannot export to DOCX.")
        print(f"   Make sure Pandoc is installed and in your system's PATH.")
        print(f"   In Colab, ensure '!apt-get install pandoc -y' was run successfully.")
        print(f"   Error details: {e}")
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH ERROR] --- 📄")
        return False
    except Exception as e:
        print(f"❌ ERROR during DOCX export: {e}")
        import traceback
        traceback.print_exc()
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH ERROR] --- 📄")
        return False

# --- Main Agentic Flow Execution ---
def run_essay_maker_flow(topic):
    print(f"\n🌟🌟🌟 STARTING ESSAY MAKER FLOW for topic: '{topic}' 🌟🌟🌟")

    # 1. Agent Planner
    main_arguments = agent_planner(topic)
    if not main_arguments or len(main_arguments) != 3:
        print("🛑 CRITICAL ERROR: Planner failed. Aborting essay creation.")
        return "Essay generation failed: Planner did not produce 3 valid arguments."

    # 2. Agent Opening Paragraph Maker
    opening_paragraph = agent_opening_paragraph_maker(topic, main_arguments)
    if not opening_paragraph:
        print("🛑 CRITICAL ERROR: Opening Paragraph Maker failed. Aborting essay creation.")
        return "Essay generation failed: Could not create opening paragraph."

    # 3. Agent Argument Paragraph Makers
    argument_paragraphs = []
    for i, arg_text in enumerate(main_arguments):
        p_num = i + 1
        paragraph = agent_argument_paragraph_maker(topic, arg_text, p_num)
        if not paragraph:
            print(f"🛑 CRITICAL ERROR: Argument Paragraph Maker for arg {p_num} failed. Aborting essay creation.")
            return f"Essay generation failed: Could not create paragraph for argument {p_num}."
        argument_paragraphs.append(paragraph)

    if len(argument_paragraphs) != 3:
        print("🛑 CRITICAL ERROR: Did not generate 3 argument paragraphs. Aborting essay creation.")
        return "Essay generation failed: Incorrect number of argument paragraphs generated."

    # 4. Agent Flow Improver (Transitions)
    transitions = agent_flow_improver(topic)
    if not transitions or len(transitions) != 4:
        print("🛑 CRITICAL ERROR: Flow Improver failed to generate 4 valid transitions. Aborting essay creation.")
        return "Essay generation failed: Could not create paragraph transitions."

    # 5. Agent Closing Paragraph Maker
    closing_paragraph = agent_closing_paragraph_maker(topic, main_arguments)
    if not closing_paragraph:
        print("🛑 CRITICAL ERROR: Closing Paragraph Maker failed. Aborting essay creation.")
        return "Essay generation failed: Could not create closing paragraph."

    # 6. Agent Essay Combiner & Markdown Formatter
    final_markdown_essay = agent_essay_combiner_markdown(
        topic,
        opening_paragraph,
        argument_paragraphs,
        closing_paragraph,
        transitions
    )

    print("\n🌟🌟🌟 ESSAY MAKER FLOW COMPLETED 🌟🌟🌟")
    return final_markdown_essay

# --- Example Usage ---
if __name__ == "__main__":
    # This is the "Input dari manusia: topik buat essay"
    user_topic = "UI adalah kampus nomor 1 di Indonesia"
    # user_topic = "The Importance of Biodiversity Conservation"
    # user_topic = "Challenges and Opportunities of Remote Work"

    print(f"User input essay topic: '{user_topic}'")

    final_markdown_output = ""
    if OPENAI_API_KEY == "DUMMY_KEY_IF_NOT_FOUND" or not OPENAI_API_KEY:
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! OpenAI API Key is missing or invalid.                  !!!")
        print("!!! The script will not call the OpenAI API.               !!!")
        print("!!! Please ensure 'openai_key' is set in Colab secrets   !!!")
        print("!!! or OPENAI_API_KEY environment variable is set.         !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        final_markdown_output = "Error: OpenAI API Key not configured. Essay generation skipped."
    else:
        # Run the agentic flow
        final_markdown_output = run_essay_maker_flow(user_topic)

    # This is the "1 time answer to chatgpt" (i.e., the final output of the flow)
    print("\n\n📜📜📜 --- FINAL ESSAY (MARKDOWN) --- 📜📜📜")
    print(final_markdown_output)
    print("\n📜📜📜 --- END OF MARKDOWN ESSAY --- 📜📜📜")

    # Export to DOCX
    if not final_markdown_output.startswith("Error:"):
        docx_filename = f"{user_topic.lower().replace(' ', '_').replace('.', '')}_essay.docx"
        export_to_docx(final_markdown_output, filename=docx_filename)
    else:
        print("\nSkipping DOCX export due to errors in essay generation.")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 9.1 MB/s eta 0:00:00
Reading package lists...
Building dependency tree...
Reading state information...
The following additional packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc-data
Suggested packages:
  texlive-latex-recommended texlive-xetex texlive-luatex pandoc-citeproc
  texlive-latex-extra context wkhtmltopdf librsvg2-bin groff ghc nodejs php
  python ruby libjs-mathjax libjs-katex citation-style-language-styles
The following NEW packages will be installed:
  libcmark-gfm-extensions0.29.0.gfm.3 libcmark-gfm0.29.0.gfm.3 pandoc
  pandoc-data
0 upgraded, 4 newly installed, 0 to remove and 34 not upgraded.
Need to get 20.6 MB of archives.
After this operation, 156 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libcmark-gfm0.29.0.gfm.3 amd64 0.29.0.gfm.3-3 [115 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 

Traceback (most recent call last):
  File "<ipython-input-14-fb374b03442d>", line 297, in export_to_docx
    pypandoc.ensure_pandoc_installed(quiet=True)
TypeError: ensure_pandoc_installed() got an unexpected keyword argument 'quiet'


In [15]:
# Step 1: Install necessary packages (run this cell first in Colab)
!pip install openai pypandoc python-docx --quiet
!apt-get install pandoc -y --quiet
print("OpenAI, pypandoc, python-docx, and pandoc installed/updated.")

import os
import re
from openai import OpenAI
import pypandoc # For Markdown to DOCX conversion

# For Google Colab secrets
# Make sure you have a secret named 'openai_key' in your Colab environment
try:
    from google.colab import userdata
    OPENAI_API_KEY = userdata.get('openai_key')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found in Colab secrets. Please add it as 'openai_key'.")
    print("Successfully loaded OpenAI API key from Colab secrets.")
except ImportError:
    # Fallback for local execution (set an environment variable OPENAI_API_KEY)
    OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')
    if OPENAI_API_KEY is None:
        raise ValueError("OpenAI API key not found. Please set the OPENAI_API_KEY environment variable or run in Colab with the secret.")
    print("Successfully loaded OpenAI API key from environment variable.")
except ValueError as e:
    print(f"Error: {e}")
    OPENAI_API_KEY = "DUMMY_KEY_IF_NOT_FOUND"

# Initialize OpenAI client
client = OpenAI(api_key=OPENAI_API_KEY)
MODEL_NAME = "gpt-4o"

# --- Helper Function for OpenAI API Calls ---
def call_openai_api(user_prompt, system_prompt=None, model=MODEL_NAME):
    print(f"    📞 Calling OpenAI API (Model: {model})...")

    messages = []
    if system_prompt:
        messages.append({"role": "system", "content": system_prompt})
    messages.append({"role": "user", "content": user_prompt})

    print(f"    ✉️  Messages sent to API: {messages}")

    try:
        response = client.responses.create(
            model=model,
            input=messages
        )
        generated_text = response.output_text
        print(f"    ✅ Raw API Output (first 300 chars): '{generated_text[:300]}...'")
        return generated_text.strip()
    except Exception as e:
        print(f"    ❌ ERROR during OpenAI API call: {e}")
        import traceback
        traceback.print_exc()
        return None

# --- Helper Function for Parsing Numbered Lists ---
def parse_numbered_list_from_llm(llm_output_str, expected_items=None):
    print(f"    📄 Attempting to parse LLM output: '{llm_output_str[:100]}...'")
    items = []
    if not llm_output_str:
        print("    ⚠️ LLM output is empty. Cannot parse.")
        return []

    lines = llm_output_str.strip().split('\n')
    for line in lines:
        line = line.strip()
        cleaned_line = re.sub(r"^\s*[\d\.\-\*\)]+\s*", "", line).strip()
        if cleaned_line:
            items.append(cleaned_line)

    if expected_items is not None and len(items) != expected_items:
        print(f"    ⚠️ Warning: Expected {expected_items} items, but parsed {len(items)}. Items: {items}")

    if not items and llm_output_str:
        print("    ⚠️ Standard parsing failed to extract items, but LLM output was not empty. Returning raw output as a single item list.")
        return [llm_output_str.strip()]

    print(f"    📊 Parsed items: {items}")
    return items

# --- Agent Definitions ---

def agent_planner(topic):
    print("\n🚀 --- Agent: Planner [STARTING] --- 🚀")
    print(f"🎯 Input to Planner: Essay Topic = '{topic}'")

    system_prompt = "You are an expert essay planner. Your task is to identify key arguments."
    user_prompt = (
        f"For an essay on the topic: '{topic}', please generate exactly 3 distinct main arguments. "
        "Each argument should be a single, concise sentence. "
        "Present these arguments as a numbered list, with each argument on a new line. For example:\n"
        "1. First argument.\n"
        "2. Second argument.\n"
        "3. Third argument."
    )

    raw_output = call_openai_api(user_prompt, system_prompt)

    if raw_output:
        arguments = parse_numbered_list_from_llm(raw_output, expected_items=3)
        if len(arguments) == 3:
            print(f"✅ Output from Planner (processed): {arguments}")
            print("🚀 --- Agent: Planner [FINISHED] --- 🚀")
            return arguments
        else:
            print(f"❌ Error: Planner did not return 3 arguments. Got {len(arguments)}: {arguments}")
            print("🚀 --- Agent: Planner [FINISHED WITH ERROR] --- 🚀")
            if len(arguments) > 0:
                 print(f"⚠️ Planner returned {len(arguments)} arguments, using the first 3 if available or padding.")
                 return (arguments + [f"Placeholder argument {i+1}" for i in range(len(arguments), 3)])[:3]
            return None
    else:
        print("❌ Error: Planner received no output from API.")
        print("🚀 --- Agent: Planner [FINISHED WITH ERROR] --- 🚀")
        return None

def agent_argument_paragraph_maker(topic, argument, arg_num):
    print(f"\n📝 --- Agent: Argument Paragraph Maker (Argument {arg_num}) [STARTING] --- 📝")
    print(f"🎯 Input: Topic='{topic}', Argument='{argument}'")

    system_prompt = "You are an expert essay writer. Your task is to write a well-developed paragraph for a given argument."
    user_prompt = (
        f"I am writing an essay on the topic: '{topic}'. "
        f"Please write a detailed paragraph for argument number {arg_num}. The argument is: '{argument}'. "
        "The paragraph should elaborate on this argument, provide supporting details or examples, and be well-structured. "
        "Aim for approximately 4-6 sentences. Ensure the paragraph is written in clear, formal essay style."
    )

    paragraph = call_openai_api(user_prompt, system_prompt)

    if paragraph:
        print(f"✅ Output from Argument Paragraph Maker (Argument {arg_num}):\n{paragraph}")
    else:
        print(f"❌ Error: Argument Paragraph Maker (Argument {arg_num}) received no output.")
    print(f"📝 --- Agent: Argument Paragraph Maker (Argument {arg_num}) [FINISHED] --- 📝")
    return paragraph

def agent_opening_paragraph_maker(topic, arguments_list):
    print("\n🌅 --- Agent: Opening Paragraph Maker [STARTING] --- 🌅")
    print(f"🎯 Input: Topic='{topic}', Main Arguments='{arguments_list}'")

    if not arguments_list or len(arguments_list) != 3:
        print("❌ Error: Opening Paragraph Maker requires 3 main arguments.")
        print("🌅 --- Agent: Opening Paragraph Maker [FINISHED WITH ERROR] --- 🌅")
        return None

    system_prompt = "You are an expert essay writer. Your task is to craft compelling introduction paragraphs."
    user_prompt = (
        f"Write an engaging opening paragraph for an essay on the topic: '{topic}'. "
        "This essay will explore the following three main arguments:\n"
        f"1. {arguments_list[0]}\n"
        f"2. {arguments_list[1]}\n"
        f"3. {arguments_list[2]}\n"
        "The opening paragraph should introduce the topic, provide brief context, and clearly state the essay's thesis "
        "or outline the main arguments that will be discussed. Do not write the full essay, only the opening paragraph. "
        "Ensure the paragraph is written in clear, formal essay style."
    )

    opening_paragraph = call_openai_api(user_prompt, system_prompt)

    if opening_paragraph:
        print(f"✅ Output from Opening Paragraph Maker:\n{opening_paragraph}")
    else:
        print("❌ Error: Opening Paragraph Maker received no output.")
    print("🌅 --- Agent: Opening Paragraph Maker [FINISHED] --- 🌅")
    return opening_paragraph

def agent_closing_paragraph_maker(topic, arguments_list):
    print("\n🌇 --- Agent: Closing Paragraph Maker [STARTING] --- 🌇")
    print(f"🎯 Input: Topic='{topic}', Main Arguments='{arguments_list}'")

    if not arguments_list or len(arguments_list) != 3:
        print("❌ Error: Closing Paragraph Maker requires 3 main arguments.")
        print("🌇 --- Agent: Closing Paragraph Maker [FINISHED WITH ERROR] --- 🌇")
        return None

    system_prompt = "You are an expert essay writer. Your task is to craft impactful concluding paragraphs."
    user_prompt = (
        f"Write a strong concluding paragraph for an essay on the topic: '{topic}'. "
        "The essay has already discussed the following three main arguments:\n"
        f"1. {arguments_list[0]}\n"
        f"2. {arguments_list[1]}\n"
        f"3. {arguments_list[2]}\n"
        "The concluding paragraph should summarize the key points made in the essay, restate the thesis in a new way, "
        "and offer a final thought, implication, or call to action. Do not introduce new arguments. "
        "Do not write the full essay, only the closing paragraph. Ensure the paragraph is written in clear, formal essay style."
    )

    closing_paragraph = call_openai_api(user_prompt, system_prompt)

    if closing_paragraph:
        print(f"✅ Output from Closing Paragraph Maker:\n{closing_paragraph}")
    else:
        print("❌ Error: Closing Paragraph Maker received no output.")
    print("🌇 --- Agent: Closing Paragraph Maker [FINISHED] --- 🌇")
    return closing_paragraph

def agent_flow_improver(topic):
    print("\n🔗 --- Agent: Flow Improver (Transitions) [STARTING] --- 🔗")
    print(f"🎯 Input: Topic='{topic}'")

    system_prompt = "You are an expert in academic writing, specializing in creating smooth and natural transition sentences between essay paragraphs."
    user_prompt = (
        f"For an essay on the topic: '{topic}', which will consist of an introduction, three main body paragraphs "
        "(each discussing a distinct argument), and a conclusion, please generate four specific, natural-sounding transition sentences:\n"
        "1. A sentence to smoothly connect the end of the introduction to the beginning of the first argument paragraph.\n"
        "2. A sentence to transition from the end of the first argument paragraph to the beginning of the second argument paragraph.\n"
        "3. A sentence to transition from the end of the second argument paragraph to the beginning of the third argument paragraph.\n"
        "4. A sentence to smoothly connect the end of the third argument paragraph to the beginning of the conclusion.\n"
        "Present these transition sentences as a numbered list, with each transition on a new line. Each should be a single, complete sentence. For example:\n"
        "1. Transition sentence one.\n"
        "2. Transition sentence two.\n"
        "..."
    )

    raw_output = call_openai_api(user_prompt, system_prompt)

    if raw_output:
        transitions = parse_numbered_list_from_llm(raw_output, expected_items=4)
        if len(transitions) == 4:
            print(f"✅ Output from Flow Improver (processed): {transitions}")
            print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED] --- 🔗")
            return transitions
        else:
            print(f"❌ Error: Flow Improver did not return 4 transitions. Got {len(transitions)}: {transitions}")
            print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED WITH ERROR] --- 🔗")
            if len(transitions) > 0:
                 print(f"⚠️ Flow Improver returned {len(transitions)} transitions, using what's available or padding.")
                 return (transitions + [f"Placeholder transition {i+1}" for i in range(len(transitions), 4)])[:4]
            return None
    else:
        print("❌ Error: Flow Improver received no output from API.")
        print("🔗 --- Agent: Flow Improver (Transitions) [FINISHED WITH ERROR] --- 🔗")
        return None

# --- Essay Combiner & Markdown Formatter ---
def agent_essay_combiner_markdown(topic, opening_para, arg_paras_list, closing_para, transitions_list):
    print("\n🧩 --- Agent: Essay Combiner & Markdown Formatter [STARTING] --- 🧩")

    if not all([topic, opening_para, arg_paras_list, closing_para, transitions_list]):
        print("❌ Error: Missing one or more parts for essay combination.")
        print(f"Debug Info: Topic: {bool(topic)}, Opening: {bool(opening_para)}, Args: {bool(arg_paras_list)}, Closing: {bool(closing_para)}, Transitions: {bool(transitions_list)}")
        if arg_paras_list: print(f"Arg Paras Length: {len(arg_paras_list)}")
        if transitions_list: print(f"Transitions Length: {len(transitions_list)}")
        print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED WITH ERROR] --- 🧩")
        return "Error: Could not combine essay due to missing parts."

    if len(arg_paras_list) != 3:
        print(f"❌ Error: Expected 3 argument paragraphs, got {len(arg_paras_list)}.")
        print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED WITH ERROR] --- 🧩")
        return "Error: Incorrect number of argument paragraphs."

    if len(transitions_list) != 4:
        print(f"❌ Error: Expected 4 transition sentences, got {len(transitions_list)}.")
        print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED WITH ERROR] --- 🧩")
        return "Error: Incorrect number of transition sentences."

    essay_title = topic.title()

    markdown_essay = f"# {essay_title}\n\n"
    markdown_essay += f"{opening_para}\n\n"
    markdown_essay += f"{transitions_list[0]}\n\n"
    markdown_essay += f"{arg_paras_list[0]}\n\n"
    markdown_essay += f"{transitions_list[1]}\n\n"
    markdown_essay += f"{arg_paras_list[1]}\n\n"
    markdown_essay += f"{transitions_list[2]}\n\n"
    markdown_essay += f"{arg_paras_list[2]}\n\n"
    markdown_essay += f"{transitions_list[3]}\n\n"
    markdown_essay += f"{closing_para}"

    print("✅ Essay combined and formatted in Markdown successfully.")
    print("🧩 --- Agent: Essay Combiner & Markdown Formatter [FINISHED] --- 🧩")
    return markdown_essay

# --- DOCX Export Function (CORRECTED) ---
def export_to_docx(markdown_content, filename="essay.docx"):
    print(f"\n📄 --- Exporting to DOCX: {filename} [STARTING] --- 📄")
    if not markdown_content or markdown_content.startswith("Error:"):
        print("❌ Error: Markdown content is invalid or indicates a prior error. Skipping DOCX export.")
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH ERROR] --- 📄")
        return False

    try:
        # Ensure Pandoc is available (pypandoc raises an error if not)
        pypandoc.ensure_pandoc_installed() # Removed 'quiet=True'

        output = pypandoc.convert_text(
            markdown_content,
            'docx',
            format='md',
            outputfile=filename
        )
        # pypandoc.convert_text returns an empty string on success when outputfile is specified
        if output == "":
            print(f"✅ Successfully exported essay to {filename}")
            print(f"📄 --- Exporting to DOCX: {filename} [FINISHED] --- 📄")
            print(f"ℹ️  In Google Colab, you can find '{filename}' in the file browser on the left panel and download it.")
            return True
        else:
            print(f"⚠️ Warning/Error during DOCX conversion. Pandoc output (if any): {output}")
            print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH WARNING/ERROR] --- 📄")
            return False

    except OSError as e:
        print(f"❌ ERROR: Pandoc not found or not configured correctly. Cannot export to DOCX.")
        print(f"   Make sure Pandoc is installed and in your system's PATH.")
        print(f"   In Colab, ensure '!apt-get install pandoc -y --quiet' was run successfully.")
        print(f"   Error details: {e}")
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH ERROR] --- 📄")
        return False
    except Exception as e:
        print(f"❌ ERROR during DOCX export: {e}")
        import traceback
        traceback.print_exc()
        print(f"📄 --- Exporting to DOCX: {filename} [FINISHED WITH ERROR] --- 📄")
        return False

# --- Main Agentic Flow Execution ---
def run_essay_maker_flow(topic):
    print(f"\n🌟🌟🌟 STARTING ESSAY MAKER FLOW for topic: '{topic}' 🌟🌟🌟")

    main_arguments = agent_planner(topic)
    if not main_arguments or len(main_arguments) != 3:
        print("🛑 CRITICAL ERROR: Planner failed. Aborting essay creation.")
        return "Essay generation failed: Planner did not produce 3 valid arguments."

    opening_paragraph = agent_opening_paragraph_maker(topic, main_arguments)
    if not opening_paragraph:
        print("🛑 CRITICAL ERROR: Opening Paragraph Maker failed. Aborting essay creation.")
        return "Essay generation failed: Could not create opening paragraph."

    argument_paragraphs = []
    for i, arg_text in enumerate(main_arguments):
        p_num = i + 1
        paragraph = agent_argument_paragraph_maker(topic, arg_text, p_num)
        if not paragraph:
            print(f"🛑 CRITICAL ERROR: Argument Paragraph Maker for arg {p_num} failed. Aborting essay creation.")
            return f"Essay generation failed: Could not create paragraph for argument {p_num}."
        argument_paragraphs.append(paragraph)

    if len(argument_paragraphs) != 3:
        print("🛑 CRITICAL ERROR: Did not generate 3 argument paragraphs. Aborting essay creation.")
        return "Essay generation failed: Incorrect number of argument paragraphs generated."

    transitions = agent_flow_improver(topic)
    if not transitions or len(transitions) != 4:
        print("🛑 CRITICAL ERROR: Flow Improver failed to generate 4 valid transitions. Aborting essay creation.")
        return "Essay generation failed: Could not create paragraph transitions."

    closing_paragraph = agent_closing_paragraph_maker(topic, main_arguments)
    if not closing_paragraph:
        print("🛑 CRITICAL ERROR: Closing Paragraph Maker failed. Aborting essay creation.")
        return "Essay generation failed: Could not create closing paragraph."

    final_markdown_essay = agent_essay_combiner_markdown(
        topic,
        opening_paragraph,
        argument_paragraphs,
        closing_paragraph,
        transitions
    )

    print("\n🌟🌟🌟 ESSAY MAKER FLOW COMPLETED 🌟🌟🌟")
    return final_markdown_essay

# --- Example Usage ---
if __name__ == "__main__":
    user_topic = "UI adalah kampus nomor 1 di Indonesia" # Using your example topic
    # user_topic = "The Societal Impact of Artificial Intelligence"
    # user_topic = "The Importance of Biodiversity Conservation"

    print(f"User input essay topic: '{user_topic}'")

    final_markdown_output = ""
    if OPENAI_API_KEY == "DUMMY_KEY_IF_NOT_FOUND" or not OPENAI_API_KEY:
        print("\n!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        print("!!! OpenAI API Key is missing or invalid.                  !!!")
        print("!!! The script will not call the OpenAI API.               !!!")
        print("!!! Please ensure 'openai_key' is set in Colab secrets   !!!")
        print("!!! or OPENAI_API_KEY environment variable is set.         !!!")
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
        final_markdown_output = "Error: OpenAI API Key not configured. Essay generation skipped."
    else:
        final_markdown_output = run_essay_maker_flow(user_topic)

    print("\n\n📜📜📜 --- FINAL ESSAY (MARKDOWN) --- 📜📜📜")
    print(final_markdown_output)
    print("\n📜📜📜 --- END OF MARKDOWN ESSAY --- 📜📜📜")

    if not final_markdown_output.startswith("Error:"):
        # Sanitize topic for filename
        safe_topic_for_filename = re.sub(r'[^\w\s-]', '', user_topic.lower()) # Remove non-alphanumeric (except space, hyphen)
        safe_topic_for_filename = re.sub(r'[-\s]+', '_', safe_topic_for_filename).strip('_') # Replace space/hyphen with underscore
        docx_filename = f"{safe_topic_for_filename}_essay.docx"
        if not docx_filename or docx_filename == "_essay.docx": # Handle empty or only suffix
            docx_filename = "generated_essay.docx"

        export_to_docx(final_markdown_output, filename=docx_filename)
    else:
        print("\nSkipping DOCX export due to errors in essay generation.")

Reading package lists...
Building dependency tree...
Reading state information...
pandoc is already the newest version (2.9.2.1-3ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.
OpenAI, pypandoc, python-docx, and pandoc installed/updated.
Successfully loaded OpenAI API key from Colab secrets.
User input essay topic: 'UI adalah kampus nomor 1 di Indonesia'

🌟🌟🌟 STARTING ESSAY MAKER FLOW for topic: 'UI adalah kampus nomor 1 di Indonesia' 🌟🌟🌟

🚀 --- Agent: Planner [STARTING] --- 🚀
🎯 Input to Planner: Essay Topic = 'UI adalah kampus nomor 1 di Indonesia'
    📞 Calling OpenAI API (Model: gpt-4o)...
    ✉️  Messages sent to API: [{'role': 'system', 'content': 'You are an expert essay planner. Your task is to identify key arguments.'}, {'role': 'user', 'content': "For an essay on the topic: 'UI adalah kampus nomor 1 di Indonesia', please generate exactly 3 distinct main arguments. Each argument should be a single, concise sentence. Present these arguments as a numbere